You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan on Artificial Intelligence

1. Introduction
- Brief overview of Artificial Intelligence (AI)
- Importance of AI in various industries
- Transition to discussing latest trends and key players in the AI field

2. Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence
- Trends: 
  - Machine learning and deep learning advancements
  - AI ethics and transparency
  - AI in healthcare and finance sectors
- Key Players:
  - Google AI
  - IBM Watson
  - Amazon AI
- Noteworthy News:
  

I now can give a great answer

Final Answer:
# The Rise of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has significantly impacted various industries, transforming the way we interact with technology and the world around us. From personalized recommendations on streaming platforms to advanced medical diagnostics, AI has revolutionized our daily lives in unprecedented ways. Exploring the latest trends and key players in the AI field is crucial to comprehending the importance of this technology and its implications on society.

## Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence
In the realm of AI, advancements in machine learning and deep learning have been pivotal, allowing computers to learn from data and make decisions autonomously. This progress has led to groundbreaking innovations across sectors such as healthcare and finance. Additionally, there has been a growing focus on AI ethics and transparency, pr

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has significantly impacted various industries, transforming the way we interact with technology and the world around us. From personalized recommendations on streaming platforms to advanced medical diagnostics, AI has revolutionized our daily lives in unprecedented ways. Exploring the latest trends and key players in the AI field is crucial to comprehending the importance of this technology and its implications on society.

## Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence
In the realm of AI, advancements in machine learning and deep learning have been pivotal, allowing computers to learn from data and make decisions autonomously. This progress has led to groundbreaking innovations across sectors such as healthcare and finance. Additionally, there has been a growing focus on AI ethics and transparency, prompting organizations to prioritize responsible AI development.

Prominent players like Google AI, IBM Watson, and Amazon AI have played a significant role in driving innovation and expanding the capabilities of AI. Their contributions have not only shaped the industry but have also established benchmarks for ethical AI practices. Recent breakthroughs in AI research, coupled with its applications in addressing global challenges like climate change, underscore the vast potential of AI to drive positive change. The emergence of AI regulations and policies worldwide highlights the importance of a unified approach to AI governance.

## Target Audience Analysis
For professionals in business, technology enthusiasts, students, and researchers, AI represents a realm of unlimited opportunities and complexities. Grasping intricate AI concepts, navigating the ever-evolving AI landscape, and implementing AI solutions in their respective fields pose challenges for this diverse audience. By staying updated on the latest AI trends and advancements, individuals can make informed decisions and leverage AI technologies effectively.

## Conclusion
As AI continues to rapidly evolve, it is essential for individuals across industries to stay informed about the latest trends, key players, and significant developments in the AI landscape. By exploring ethical considerations, practical applications, and regulatory frameworks associated with AI, we can fully harness the transformative power of this technology. Let's embrace the potential of AI and work towards creating a smarter, more sustainable future.

*SEO Keywords: Artificial Intelligence, AI trends, Machine learning advancements, Key players in AI, Noteworthy AI news, AI applications, AI ethics*

*Resources: Forbes article on latest AI trends, TechCrunch report on key players in AI industry, Harvard Business Review publication on AI ethics, World Economic Forum analysis on AI applications in different sectors*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).